In [3]:
import torch
import tensorrt
import torch_tensorrt
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
from torchvision.datasets import MNIST
from torch.utils.data import random_split
import torch.nn.functional as F
import numpy as np

In [4]:
mnist_dataset = MNIST(root = 'data/',download = True, train = True, transform = transforms.ToTensor())
print(mnist_dataset)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data/
    Split: Train
    StandardTransform
Transform: ToTensor()


In [5]:
batch_size = 500
train_data, validation_data = random_split(mnist_dataset, [50000, 10000])

train_loader = DataLoader(train_data, batch_size, shuffle = True)
val_loader = DataLoader(validation_data, batch_size, shuffle = False)

In [6]:
class Amnist(nn.Module):
    def __init__(self,num_classes):
        super(Amnist, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 3, 1)
        self.conv2 = nn.Conv2d(64, 512, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.conv3 = nn.Conv2d(512, 128, 3, 1)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(12800, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
        
        

In [7]:
class AmnistClassifier:
    def __init__(self):
        self.num_epochs = 10
        self.lr = 0.005

        self.criterion = nn.CrossEntropyLoss()
        self.device = "cuda"
        self.model = Amnist(10).to(self.device)
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr = self.lr)
    
    def train_epoch(self):
        epoch_loss = 0
        for i, (images, labels) in enumerate(train_loader):

            images = images.to(self.device)
            labels = labels.to(self.device)
            # forward pass
            out = self.model(images)
            loss = self.criterion(out, labels)
            epoch_loss += loss

            # backward pass
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        return epoch_loss / len(train_loader)
    
    def accuracy(self,outputs, labels):
        _, preds = torch.max(outputs, dim = 1)
        return(torch.tensor(torch.sum(preds == labels).item()/ len(preds)))
    
    def get_val_loss(self):
        epoch_loss = 0
        acc = []
        with torch.no_grad():
            for i, (images, labels) in enumerate(val_loader):

                images = images.to(self.device)
                labels = labels.to(self.device)

                # forward pass
                out = self.model(images)
                loss = self.criterion(out, labels)
                epoch_loss += loss
                acc.append(self.accuracy(out,labels))
                
        print(f'Accuracy: {np.mean(acc):.4f}')
        return epoch_loss / len(val_loader)
    
    
    
    def train(self,num_epochs):
        train_losses = []
        val_losses = []
        best_score = float("inf")
        tol = 0

        for epoch in range(num_epochs):
            print(f"epoch {epoch+1}:")

            self.model.train(True)
            avg_train_epoch_loss = self.train_epoch()

            self.model.eval()
            avg_val_epoch_loss = self.get_val_loss()

            if(avg_val_epoch_loss < best_score):
                best_score = avg_val_epoch_loss
                print("Saving ...")
                torch.save(self.model.state_dict(), f"./saves/best.pt")
            else:
                tol += 1
                if(tol == 4):
                    break


            train_losses.append(avg_train_epoch_loss)
            val_losses.append(avg_val_epoch_loss)


            print(f"epoch: {epoch+1} --- avg_train_loss = {avg_train_epoch_loss} --- avg_val_loss = {avg_val_epoch_loss}") 
            print("#"*100)
     
    
    

In [6]:
classifier = AmnistClassifier()
classifier.train(10)
        

epoch 1:
Accuracy: 0.3211
Saving ...
epoch: 1 --- avg_train_loss = 2.290242910385132 --- avg_val_loss = 2.2692174911499023
####################################################################################################
epoch 2:
Accuracy: 0.5393
Saving ...
epoch: 2 --- avg_train_loss = 2.228464126586914 --- avg_val_loss = 2.1506412029266357
####################################################################################################
epoch 3:
Accuracy: 0.7355
Saving ...
epoch: 3 --- avg_train_loss = 1.9176583290100098 --- avg_val_loss = 1.443790078163147
####################################################################################################
epoch 4:
Accuracy: 0.8299
Saving ...
epoch: 4 --- avg_train_loss = 1.0873442888259888 --- avg_val_loss = 0.6581984758377075
####################################################################################################
epoch 5:
Accuracy: 0.8619
Saving ...
epoch: 5 --- avg_train_loss = 0.6919280290603638 --- avg_val_loss 

In [8]:
import time 
import numpy as np

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim = 1)
    return(torch.tensor(torch.sum(preds == labels).item()/ len(preds)))
    
def infer(model,loader):
    epoch_loss = 0
    acc = []
    device = "cuda"
    fps_arr = []
    with torch.no_grad():
        for i, (images, labels) in enumerate(loader):
            images = images.to(device)
            labels = labels.to(device)
            if images.shape[0] !=500:
                continue
            # forward pass
            start = time.time()
            out = model(images)
            fps_arr.append(1/(time.time()-start))
            acc.append(accuracy(out,labels))

    print(f'Accuracy: {np.mean(acc):.4f}, AVG FPS: {np.mean(fps_arr)}')
    return np.mean(acc)

In [9]:
classifier = AmnistClassifier()
classifier.model.load_state_dict(torch.load("./saves/best.pt"))
classifier.model.eval()


Amnist(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(64, 512, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (conv3): Conv2d(512, 128, kernel_size=(3, 3), stride=(1, 1))
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=12800, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [10]:
infer(classifier.model.to("cuda"),val_loader)

Accuracy: 0.9102, AVG FPS: 3686.2208595916686


0.91019994

In [11]:
model = classifier.model.eval().to("cuda")

In [12]:
script = torch.jit.script(model)

In [13]:
trt_model_fp32 = torch_tensorrt.compile(script, inputs = [torch_tensorrt.Input((500, 1, 28, 28), dtype=torch.float32)],
    enabled_precisions = {torch.float32}, # Run with FP32
    workspace_size = 1 << 22
)
#torch.jit.save(trt_model_fp32, "trt_model.ts")

In [14]:
infer(trt_model_fp32,val_loader)

Accuracy: 0.9102, AVG FPS: 10822.715728265137


0.91019994